# Sentiment Analysis Prediction

## Dependecies

In [ ]:
!pip install neattext

In [ ]:
import numpy as np
import pandas as pd
import spacy
import neattext.functions as nfx
import matplotlib.pyplot as plt
import seaborn as sns

## Cargar Dataset

In [ ]:
from google.colab import files
files.upload()

In [ ]:
df = pd.read_csv('Sentiment Analysis Dataset (1).csv', encoding='latin-1')

In [ ]:
df.head()

In [ ]:
df.shape

## Data Cleaning

In [ ]:
df['text_clean'] = df['SentimentText'].apply(nfx.remove_punctuations)

In [ ]:
df['text_clean'] = df['text_clean'].apply(nfx.remove_shortwords)

In [ ]:
df['text_clean'] = df['text_clean'].apply(nfx.remove_special_characters)

In [ ]:
df['text_clean'] = df['text_clean'].apply(nfx.remove_puncts)

In [ ]:
df['text_clean'] = df['text_clean'].apply(nfx.remove_dates)

In [ ]:
df[['SentimentText', 'text_clean']].head()

## Crear polaridad y Subjetividad  
Métricas de polaridad y subjetividad
Para comenzar nuestro viaje, veamos la oferta de TextBlob. Con TextBlob, obtenemos una métrica de polaridad y subjetividad. La polaridad es el sentimiento mismo, que va de -1 a +1. La subjetividad es una medida del sentimiento siendo objetivo a subjetivo, y va de 0 a 1

In [ ]:
from textblob import TextBlob

In [ ]:
df['polarity'] = df['text_clean'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [ ]:
df['subjectivity'] = df['text_clean'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

In [ ]:
df.head()

In [ ]:
#Get sentiment labels
def get_sentiment(polarity):
  if polarity >0:
    label = 'Positive'
  elif polarity <0:
    label ='Negative'
  else:
    label = 'Neutral'
  return label

In [ ]:
df['sentiment_label'] = df['polarity'].apply(lambda x: get_sentiment(x))

In [ ]:
df.head()

In [ ]:
df['sentiment_label'].value_counts()

In [ ]:
sns.countplot(df['sentiment_label'])
plt.show()

In [ ]:
print('Porcentaje:')
round(df['sentiment_label'].value_counts(normalize = True)*100, 2)

### Reducir los datos a 1000 registros

In [ ]:
df1 = df[:1000]
df1.head()

In [ ]:
df1.shape

## Sentiment Scores

In [ ]:
!pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
def getSIA(text):
  sia = SentimentIntensityAnalyzer()
  sentiment= sia.polarity_scores(text)
  return sentiment

In [ ]:
compound = []
neg = []
neu = []
pos = []

for i in range(0, len(df1['text_clean'])):
    SIA = getSIA(df1['text_clean'][i])
    compound.append(SIA['compound'])
    neg.append(SIA['neg'])
    neu.append(SIA['neu'])
    pos.append(SIA['pos'])

In [ ]:
df1['compound'] = compound
df1['negative'] = neg
df1['positive'] = pos
df1['neutral'] = neu

In [ ]:
df1.head()

In [ ]:
df1.to_csv('sent_clasif.csv', index=False)

In [ ]:
df1.columns

### Feature selection

In [ ]:
keep_columns = ['Sentiment',  'polarity', 'subjectivity', 'compound', 'negative', 'positive', 'neutral']
data = df1[keep_columns]
data.head()

In [ ]:
X = data.drop('Sentiment', axis =1)
y = data['Sentiment']
X.shape, y.shape

Split into Train and Test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state= 42)

Build Model

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
model = LinearDiscriminantAnalysis()
model.fit(X_train, y_train)

In [ ]:
yhat = model.predict(X_test)

In [ ]:
result = pd.DataFrame({'actual': y_test, 'predicted': yhat})
result.head()

In [ ]:
result['diferencia'] = result['actual'] - result['predicted']

In [ ]:
result['diferencia'].value_counts()

In [ ]:
result['diferencia'].value_counts().plot(kind = 'barh')
plt.show()

In [ ]:
result[result['diferencia']>0].head()

In [ ]:
result[result['diferencia']<0]

### score

In [ ]:
from sklearn.metrics import  classification_report

In [ ]:
print(classification_report(y_test, yhat))